In [11]:
import math
import numpy as np
import wandb

import torch
import torch_geometric
from torch_geometric.data import Data

import sys
import os
from tqdm import tqdm

# Add the 'scripts' directory to the Python path
scripts_path = os.path.abspath(os.path.join('..'))
if scripts_path not in sys.path:
    sys.path.append(scripts_path)

# Now you can import the gnn_io module
import gnn_io as gio

import gnn_architectures as garch

## 1. Define model and parameters

In [12]:
wandb.login()

# Define parameters 
num_epochs = 1000
project_name = 'gnn_target_normalized_features_car_vol_baseline_capacity_reduction_and_highway'

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

early_stopping = gio.EarlyStopping(patience=10, verbose=True)
torch.set_printoptions(precision=4, sci_mode=False)

wandb: WARNING Calling wandb.login() after wandb.init() has no effect.


## 2. Load data

In [13]:
# Load the list of dictionaries
data_dict_list = torch.load('../../data/train_data/dataset_1pm_0-2600_16_07.pt')

# Reconstruct the Data objects
datalist = [Data(x=d['x'], edge_index=d['edge_index'], pos=d['pos'], y=d['y']) for d in data_dict_list]

In [14]:
dataset_only_relevant_dimensions = gio.cut_dimensions(dataset=datalist, indices_of_dimensions_to_keep=[0, 1, 2, 3])
dataset_normalized = gio.normalize_dataset(dataset_only_relevant_dimensions)

In [15]:
baseline_error = gio.compute_baseline_of_no_policies(dataset_normalized)
print(f'Baseline error: {baseline_error}')

baseline_error = gio.compute_baseline_of_mean_target(dataset_normalized)
print(f'Baseline error: {baseline_error}')

Baseline error: 0.32162734866142273
mean_y_normalized: 
0.56424004
median_y_normalized: 
0.56032723
Baseline error: 0.0032576548401266336


## 4. Train the model

We first find a good model for one batch. 

In [16]:
def train(model, config=None, loss_fct=None, optimizer=None, train_dl=None, valid_dl=None, device=None, early_stopping=None):
    for epoch in range(config.epochs):
        model.train()
        for idx, data in tqdm(enumerate(train_dl)):
            input_node_features, targets = data.x.to(device), data.y.to(device)
            optimizer.zero_grad()

            # Forward pass
            predicted = model(data)
            train_loss = loss_fct(predicted, targets)
            
            # Backward pass
            train_loss.backward()
            optimizer.step()
            
            wandb.log({"train_loss": train_loss.item(), "epoch": epoch, "step": idx})
            # print(f"epoch: {epoch}, step: {idx}, loss: {train_loss.item()}")
        
        val_loss = garch.validate_model_pos_features(model, valid_dl, loss_fct, device)
        print(f"epoch: {epoch}, validation loss: {val_loss}")
        wandb.log({"loss": val_loss, "epoch": epoch})
            
        early_stopping(val_loss)
        if early_stopping.early_stop:
            print("Early stopping triggered. Stopping training.")
            break
    
    print("Best validation loss: ", val_loss)
    wandb.summary["val_loss"] = val_loss
    wandb.finish()
    return val_loss, epoch

In [17]:
batch_size = 16
train_dl, valid_dl, test_dl = gio.create_dataloaders(batch_size = batch_size, dataset=dataset_normalized, train_ratio=0.7, val_ratio=0.15, test_ratio=0.15)

Total dataset length: 2580
Training subset length: 1805
Validation subset length: 387
Test subset length: 388


In [18]:
gio.save_dataloader(test_dl, 'test_dl_new_2.pt')
gio.save_dataloader_params(test_dl, 'test_loader_params_new_2.json')

In [19]:
# Currently best architecture ! 
output_layer_parameter = 'gat'
hidden_size_parameter = 16
gat_layer_parameter = 0
gcn_layer_parameter = 0
lr = 0.0001
in_channels = 6 # dimensions of the x vector + 2 (pos)

wandb.login()

wandb.init(
    project=project_name,
    config={
        "epochs": num_epochs,
        "batch_size": batch_size,
        "lr": lr,
        "early_stopping_patience": 10,
        "hidden_layer_size": hidden_size_parameter,
        "gat_layers": gat_layer_parameter,
        "gcn_layers": gcn_layer_parameter,
        "output_layer": output_layer_parameter,
        # "dropout": 0.15,
    }
)
config = wandb.config

print("output_layer: ", output_layer_parameter)
print("hidden_size: ", hidden_size_parameter)
print("gat_layers: ", gat_layer_parameter)
print("gcn_layers: ", gcn_layer_parameter)

gnn_instance = garch.MyGnn(in_channels=in_channels, out_channels=1, hidden_size=hidden_size_parameter, gat_layers=gat_layer_parameter, gcn_layers=gcn_layer_parameter, output_layer=output_layer_parameter)
model = gnn_instance.to(device)

best_val_loss, best_epoch = train(model, config=config, 
                                loss_fct=torch.nn.MSELoss(), 
                                optimizer=torch.optim.Adam(model.parameters(), lr=lr),
                                train_dl=train_dl, valid_dl=valid_dl,
                                device=device, early_stopping=early_stopping)

wandb: WARNING Calling wandb.login() after wandb.init() has no effect.


epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇███
loss,██▇▆▆▆▅▅▅▄▄▄▃▃▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
step,▂▄▆█▂▄▆█▃▅▇▁▃▅▇▂▄▅▇▂▅▇▂▄▅▇▂▄▆█▂▄▆▁▃▅▇▁▃▆
train_loss,██▇▆▆▆▅▅▅▄▄▄▄▃▃▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
epoch,92
loss,0.00289
step,9
train_loss,0.00289


output_layer:  gat
hidden_size:  16
gat_layers:  0
gcn_layers:  0
Model initialized
MyGnn(
  (pointLayer): PointNetConv(local_nn=Sequential(
    (0): Linear(in_features=6, out_features=16, bias=True)
    (1): ReLU()
    (2): Linear(in_features=16, out_features=16, bias=True)
  ), global_nn=Sequential(
    (0): Linear(in_features=16, out_features=8, bias=True)
    (1): ReLU()
    (2): Linear(in_features=8, out_features=32, bias=True)
    (3): ReLU()
    (4): Linear(in_features=32, out_features=16, bias=True)
  ))
  (output_layer): GATConv(16, 1, heads=1)
)


113it [12:53,  6.84s/it]


epoch: 0, validation loss: 0.1439522510766983


...

epoch: 463, validation loss: 0.002932921750470996
EarlyStopping counter: 1 out of 10


113it [01:55,  1.03s/it]


epoch: 464, validation loss: 0.0029326747078448536


11it [00:10,  1.02it/s]


KeyboardInterrupt: 

In [ ]:
# torch.save(model, '../../data/trained_models/model_16_07_new.pth')

In [ ]:
model_path = '../../data/trained_models/model_16_07_lr_0.0001.pth'

# Save the model state dictionary and configuration
torch.save({
    'state_dict': model.state_dict(),
    'config': {
        'in_channels': model.in_channels,
        'out_channels': 1,
        'hidden_size': model.hidden_size,
        'gat_layers': model.gat_layers,
        'gcn_layers': model.gcn_layers,
        'output_layer': model.output_layer
    }
}, model_path)